In [36]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-122639")
exp = Experiment(workspace=ws, name="Projectone")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-122639
Azure region: southcentralus
Subscription id: 94e14ad4-bf97-47e8-aae0-f9b85a7befa8
Resource group: aml-quickstarts-122639


In [43]:
from azureml.core.run import Run
run_logger = Run.get_context()
run_logger.log("accuracy",float(val_accuracy))

NameError: name 'val_accuracy' is not defined

In [37]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "cpu-cluster-1"
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Cluster exists, it can be used.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Cluster exists, it can be used.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [46]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import normal, uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": choice(0.01,0.1,1,10,100,1000),
    "batch_size": choice(16, 32, 64, 128,256),
    "--max_iter": choice(5,10,50,100,500)
})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./',compute_target=compute_target,entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Loss",
    max_concurrent_runs=5,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=50)

In [47]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
exp_tuning = Experiment(workspace=ws, name='hypetuning')

run = exp_tuning.submit(hyperdrive_config)
run

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


Experiment,Id,Type,Status,Details Page,Docs Page
hypetuning,HD_8fb6b439-7e88-4066-aedc-df777ea0407c,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [48]:
#Showing details of the run
RunDetails(run).show()
run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_8fb6b439-7e88-4066-aedc-df777ea0407c
Web View: https://ml.azure.com/experiments/hypetuning/runs/HD_8fb6b439-7e88-4066-aedc-df777ea0407c?wsid=/subscriptions/94e14ad4-bf97-47e8-aae0-f9b85a7befa8/resourcegroups/aml-quickstarts-122639/workspaces/quick-starts-ws-122639

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-10-24T17:23:50.625765][API][INFO]Experiment created<END>\n""<START>[2020-10-24T17:23:51.378151][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n"<START>[2020-10-24T17:23:51.7036564Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-10-24T17:23:56.026237][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"


ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [25]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
b_r = run.get_best_run_by_primary_metric()
if b_r is None:
    raise Exception("None found")
b_r

Exception: None found

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
file_path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
data_set = Dataset.Tabular.from_delimited_files(path=file_path)

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(data_set)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.core.experiment import Experiment

experiment = Experiment(ws, "banking_experiment")
run = experiment.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
from azureml.core.model import Model
import os

model = Model(workspace=ws, name="banking_model")
model.download(target_dir=os.getcwd())